prompt：
請生成一個 Python 程式碼，功能如下：

下載與解壓縮數據：

從指定的 Google Drive 連結下載一個壓縮檔（如 data.zip），解壓到當前目錄中的一個資料夾（如 data）。
若檔案已存在，跳過下載和解壓步驟。
讀取資料：

從解壓後的資料夾中讀取兩個 CSV 檔案（如 train.csv 和 test.csv），並將其載入 Pandas 資料框。
特徵工程：

將資料進行以下轉換：
把年齡（Age）按年齡段分箱（如嬰兒、少年、成人等）。
抽取艙位（Cabin）的首字母作為新特徵，填補缺失值。
把票價（Fare）按四分位數進行分箱處理。
從姓名（Name）中提取姓氏（Lname）和稱謂（NamePrefix）。
刪除不必要的欄位（如 Ticket, Name, Embarked）。
資料分割：

將訓練集（train.csv）按 8:2 的比例分割成訓練資料和測試資料。
機器學習模型訓練與比較：

使用 pycaret.classification 模組進行以下操作：
設置目標變數為 Survived。
訓練至少 16 種不同的分類模型，包括邏輯回歸（lr）、隨機森林（rf）、梯度提升（xgboost）等。
堆疊這些模型，使用 xgboost 作為元模型。
模型評估：

使用測試資料進行預測，計算並輸出模型的準確率。
給我完整的程式碼

In [2]:
import os
import zipfile
import requests
import pandas as pd

# 定義下載和解壓函數
def download_and_extract(url, zip_file_name, extracted_folder):
    if not os.path.exists(zip_file_name):
        response = requests.get(url, stream=True)
        with open(zip_file_name, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print(f"Downloaded: {zip_file_name}")
    if not os.path.exists(extracted_folder):
        with zipfile.ZipFile(zip_file_name, "r") as zip_ref:
            zip_ref.extractall(extracted_folder)
        print(f"Extracted to: {extracted_folder}")

# 下載並解壓縮資料
url = "https://drive.google.com/uc?id=1ZfxPkqBfZa4SFVPbwmyghbLiiKNUqfCj"
zip_file_name = "data.zip"
extracted_folder = "data"
download_and_extract(url, zip_file_name, extracted_folder)

# 讀取資料
data_train = pd.read_csv(os.path.join(extracted_folder, "train.csv"))
data_test = pd.read_csv(os.path.join(extracted_folder, "test.csv"))

# 資料轉換函數
def transform_features(df):
    df['Age'] = pd.cut(df['Age'].fillna(-0.5),
                       bins=(-1, 0, 5, 12, 18, 25, 35, 60, 120),
                       labels=['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior'])
    df['Cabin'] = df['Cabin'].fillna('N').apply(lambda x: x[0])
    df['Fare'] = pd.cut(df['Fare'].fillna(-0.5),
                        bins=(-1, 0, 8, 15, 31, 1000),
                        labels=['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile'])
    df['Lname'] = df['Name'].apply(lambda x: x.split(' ')[0])
    df['NamePrefix'] = df['Name'].apply(lambda x: x.split(' ')[1])
    df = df.drop(['Ticket', 'Name', 'Embarked'], axis=1)
    return df

# 應用轉換
data_train = transform_features(data_train)
data_test = transform_features(data_test)

# 切分資料集
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_train.drop(['PassengerId'], axis=1),
                                         random_state=100, train_size=0.8)

# 安裝 PyCaret
!pip install pycaret scikit-learn

# 引入 PyCaret 並建立模型
from pycaret.classification import setup, compare_models, create_model, stack_models, save_model, load_model, predict_model
clf1 = setup(data=train_data,
             target='Survived',
             categorical_features=['Pclass', 'Sex', 'Age', 'Fare', 'Cabin', 'Lname', 'NamePrefix'])

# 比較和創建16個模型
model_list = [
    'lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'gpc',
    'mlp', 'ridge', 'rf', 'qda', 'ada', 'lda', 'gbc',
    'et', 'xgboost', 'gpc'
]
models = {name: create_model(name, fold=5) for name in model_list}

# 堆疊模型
meta_model = create_model('xgboost', fold=5)
stacker_all = stack_models(estimator_list=list(models.values()), meta_model=meta_model)

# 預測並計算準確率
pred = predict_model(stacker_all, data=test_data)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(pred['Survived'], pred['prediction_label'])
print(f"Stacked Model Accuracy: {accuracy}")


,Description,Value
0,Session id,5418
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(712, 10)"
4,Transformed data shape,"(712, 52)"
5,Transformed train set shape,"(498, 52)"
6,Transformed test set shape,"(214, 52)"
7,Numeric features,2
8,Categorical features,7
9,Preprocess,True


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8100,0.8623,0.6579,0.8065,0.7246,0.5819,0.5889
1,0.7900,0.8256,0.7105,0.7297,0.7200,0.5520,0.5522
2,0.8400,0.8790,0.7568,0.8000,0.7778,0.6529,0.6535
3,0.8081,0.8459,0.6486,0.8000,0.7164,0.5738,0.5810
4,0.7980,0.8470,0.5946,0.8148,0.6875,0.5436,0.5583
Mean,0.8092,0.8519,0.6737,0.7902,0.7253,0.5808,0.5868
Std,0.0170,0.0179,0.0555,0.0307,0.0293,0.0386,0.0361


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7900,0.8130,0.6053,0.7931,0.6866,0.5329,0.5439
1,0.7500,0.8304,0.6053,0.6970,0.6479,0.4556,0.4583
2,0.8100,0.8166,0.6486,0.8000,0.7164,0.5759,0.5831
3,0.8182,0.8219,0.6486,0.8276,0.7273,0.5939,0.6038
4,0.8182,0.8509,0.6486,0.8276,0.7273,0.5939,0.6038
Mean,0.7973,0.8266,0.6313,0.7890,0.7011,0.5504,0.5586
Std,0.0258,0.0135,0.0213,0.0481,0.0305,0.0524,0.0547


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7200,0.7725,0.5263,0.6667,0.5882,0.3805,0.3866
1,0.7000,0.6469,0.5263,0.6250,0.5714,0.3433,0.3463
2,0.8400,0.8854,0.7895,0.7895,0.7895,0.6604,0.6604
3,0.7000,0.7343,0.4737,0.6429,0.5455,0.3292,0.3377
4,0.7600,0.8302,0.5789,0.7333,0.6471,0.4690,0.4766
5,0.7400,0.7903,0.6316,0.6667,0.6486,0.4425,0.4429
6,0.6200,0.6664,0.3158,0.5000,0.3871,0.1316,0.1389
7,0.7800,0.8038,0.6667,0.7059,0.6857,0.5167,0.5172
8,0.8163,0.7948,0.6667,0.8000,0.7273,0.5905,0.5961


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.7877,0.8372,0.6400,0.8136,0.7164,0.5506,0.5607


Stacked Model Accuracy: 0.7877094972067039
